# Anexo B - Python: Confianza ($\epsilon$) vs Núm Threads

### Paquetes

In [1]:
#import threading
import time
import numpy as np
import pandas as pd
import multiprocessing as mp

### Funciones

In [2]:
def elementos_aux(A,b,δ):
    
    #Construcción de elementos auxiliares del sistema Ax = b.
    #δ := precisión.
    
    #M es la matriz diagonal de A
    M = np.diag(np.diag(A))
    
    #invM es la inversa de M
    invM = np.linalg.inv(M)
    
    #A = M-N (factorización de A)
    N = M - A
    
    #Matriz auxiliar T
    T = np.dot(invM,N)
    
    #Matriz auxiliar F
    f = np.dot(invM, b)
    
    #Dimensiones de la matriz T
    nT, mT = T.shape
    
    #Matriz de Probabilidades de transición
    ProbT = (T != 0)/np.apply_along_axis(sum,1,T != 0)[:,None]
    
    #Probailidades de los estados iniciales
    Pi=(np.apply_along_axis(sum,1,T!=0)!=0)/sum((np.apply_along_axis(sum,1,T!=0)!=0))
    
    #Número de cadenas
    NormT = np.linalg.norm(T)
    NormF = np.linalg.norm(f)
    N = np.floor(np.power(0.6745/δ,2)*np.power(NormF,2)/np.power(1-NormT,2))+1
    N = int(N)
    
    return f, N, nT, T, Pi, ProbT

In [3]:
def MarkovChain(ϵ,i,f,P,T):
    
    #Simulando el estimados de una cadena de Markov
    
    #ϵ := Confianza.
    #i := Indice sobre las filas de T.
    #f := Matriz auxiliar f.
    #P := Matriz con las probabilidades de transición.
    #T := Matriz auxiliar T.
    
    #Inicializando las variables auxiliares
    W_0 = 1
    W = W_0
    k = 0
    point = i
    #print(P[point,:])
    X = W_0 * f[i]
    
    while np.absolute(W) >= ϵ:
        
        #Simulando el siguiente estado de la cadena
        proba = P[point,:].flatten()
        nextpoint = np.where(np.random.multinomial(1,proba) == 1)[0]
        
        #Avanzando el contador
        k = k + 1
        
        #Actualizando X
        if T[point, nextpoint] != 0:
            W_new = W *(T[point, nextpoint]/P[point, nextpoint])
            X = X + W_new * f[nextpoint]
                   
        #Actualizando la fila
        point = nextpoint
        W = W_new
    
    return X[0]

In [4]:
def estimador(ϵ,i,f,ProbT,T,N):
    return np.mean([MarkovChain(ϵ,i,f,ProbT,T) for s in range(0,N)])

In [5]:
def MCMCPar(A,b,ϵ,δ,Nh,M = 0):
    
    #Estimación de la solución del sistema de ec. Ax = b.
    
    #A := Matriz de coeficientes del sistema.
    #b := Vector de constantes.
    #ϵ := Precisión.
    #δ := Confianza.
    #Nh:= numero de hilos.
    
   
    #Construcción de elementos auxiliares del sistema Ax = b.
    f, N, nT, T, Pi, ProbT  = elementos_aux(A,b,δ)
    
    if M != 0:
        N = M
        
    #Balanceando cargas
    N = np.floor(N/Nh) + 1
    N = int(N)

    #Estimación del vector
    x = np.empty([nT, 1])
    #print(x.shape)
    
    if Nh == 1:
        
        for i in range(0,nT):
            #i es un indice que corre sobre las filas
            x[i] = estimador(ϵ,i,f,ProbT,T,N)
            
    else:
        
        args = [(ϵ,i,f,ProbT,T,N) for i in range(0,nT)] 
        with mp.Pool(processes=Nh) as pool:
            x = pool.starmap(estimador, args)
            
    return x

### Construcción del sistema de ecuaciones Ax = b.

In [6]:
def MatrixCoef(tam):
    
    #tam := Tamaño del sistema EQ
    
    #Matriz de coeficientes
    #Construyendo las diagonales
    A = np.diag([3 for i in range(0,tam)], k = 0)
    A = A - np.diag([1 for i in range(0,tam-1)], k = 1)
    A = A - np.diag([1 for i in range(0,tam-1)], k = -1)
    
    #Construyendo la anti-diagonal mayor
    B = np.diag([0.5 for i in range(0,tam)])
    i = int(np.floor(tam/2))
    B[i,i] = 0
    if tam % 2 == 0:
        B[i-1,i-1] = 0
    
    B = np.fliplr(B)
    
    return A + B

## Experimento

#### Tamaño de la entrada (dimensión de la matriz A)

In [7]:
tam = 20

#### Número de simulaciones

In [8]:
N = 100

#### Simulaciones

In [9]:
id_sim = [i for i in range(0,3600)]
ϵ = [e for e in [0.05, 0.06, 0.07, 0.08, 0.09, 0.10] for i in range(0,600)]
nthreads = [nt for i in range(0,600) for nt in [1,2,4,8,16,32] ] 
tiempo = [0.00 for i in range(0,3600)]
data = [id_sim,ϵ,nthreads,tiempo]
data = pd.DataFrame(data)
data = data.transpose()
data.columns = ["Id","Confianza","Nthreads","Tiempo"]
data

,Id,Confianza,Nthreads,Tiempo
0,0.0,0.05,1.0,0.0
1,1.0,0.05,2.0,0.0
2,2.0,0.05,4.0,0.0
3,3.0,0.05,8.0,0.0
4,4.0,0.05,16.0,0.0
...,...,...,...,...
3595,3595.0,0.10,2.0,0.0
3596,3596.0,0.10,4.0,0.0
3597,3597.0,0.10,8.0,0.0
3598,3598.0,0.10,16.0,0.0


In [10]:
#Construyendo la matriz de coeficientes
A = MatrixCoef(20)

#Construyendo la solución
x = [1.0 for i in range(0,20)]
x = np.array(x)
    
#Construyendo el vector de constantes
b = np.dot(A,x)

for i in range(0,3600):
    
    #Simulaciones:

    #Tomando el tiempo inicial
    t = time.time()
    
    MCMCPar(A,b,data.Confianza[i],1,int(data.Nthreads[i]),N)
    
    #Guardando el tiempo
    data.Tiempo[i] = time.time() - t
    
    if i % 100 == 0:
        print("Fila:",i,"de 3600")
        data.to_csv("MiniP3_Python_epsilon.csv")
        
print("Fila:",i,"de 3000")    
data.to_csv("MiniP3_Python_epsilon.csv")

Fila: 0 de 3600
Fila: 100 de 3600
Fila: 200 de 3600
Fila: 300 de 3600
Fila: 400 de 3600
Fila: 500 de 3600
Fila: 600 de 3600
Fila: 700 de 3600
Fila: 800 de 3600
Fila: 900 de 3600
Fila: 1000 de 3600
Fila: 1100 de 3600
Fila: 1200 de 3600
Fila: 1300 de 3600
Fila: 1400 de 3600
Fila: 1500 de 3600
Fila: 1600 de 3600
Fila: 1700 de 3600
Fila: 1800 de 3600
Fila: 1900 de 3600
Fila: 2000 de 3600
Fila: 2100 de 3600
Fila: 2200 de 3600
Fila: 2300 de 3600
Fila: 2400 de 3600
Fila: 2500 de 3600
Fila: 2600 de 3600
Fila: 2700 de 3600
Fila: 2800 de 3600
Fila: 2900 de 3600
Fila: 3000 de 3600
Fila: 3100 de 3600
Fila: 3200 de 3600
Fila: 3300 de 3600
Fila: 3400 de 3600
Fila: 3500 de 3600
Fila: 3599 de 3000
